### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Search using Keyword and Vector Database

In [ ]:
# !pip install cohere
# !pip install weaviate-client

In [1]:
# import libraries
import weaviate
import cohere

In [2]:
# Add your Cohere API key here
# You can obtain a key by signing up in https://dashboard.cohere.com/ or https://docs.cohere.com/reference/key
cohere_api_key = ''

In [3]:
# Create a Cohere client object
co = cohere.Client(cohere_api_key)

In [4]:
# Create a vector database object
# Connect to the Weaviate demo databse containing 10M wikipedia vectors
# You can obtain a key from https://weaviate.io and refer to https://weaviate.io/developers/wcs/authentication#connect-with-an-api

auth_config = weaviate.auth.AuthApiKey(api_key="")
client = weaviate.Client(
    url="https://cohere-demo.weaviate.network/",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": cohere_api_key,
    }
)

/home/datascience/conda/generalml_p38_cpu_v1/lib/python3.8/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.1. The latest version is 4.5.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [5]:
# Keyword Function
def keyword_search(query, results_lang='en', num_results=3):
    properties = ["text", "title", "url"]


    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }


    response = (
    client.query.get("Articles", properties)
    .with_bm25(
    query=query
    )
    .with_where(where_filter)
    .with_limit(num_results)
    .do()
    )
    result = response['data']['Get']['Articles']
    return result

In [6]:
# Example usage
query = "What is the highest mountain peak in the world?"
top_documents = keyword_search(query)

In [7]:
top_documents

[{'text': "The Polish scientist and explorer Count Paul Edmund Strzelecki conducted surveying work in the Australian Alps in 1839 and, led by his two Aboriginal guides Charlie Tarra and Jackie, became the first European to ascend Australia's highest peak, which he named Mount Kosciuszko in honour of the Polish patriot Tadeusz Kościuszko.European explorers penetrated deeper into the interior in the 1840s in a quest to discover new lands for agriculture or answer scientific enquiries. The German scientist Ludwig Leichhardt led three expeditions in northern Australia in this decade, sometimes with the help of Aboriginal guides, identifying the grazing potential of the region and making important discoveries in the fields of botany and geology. He and his party disappeared in 1848 while attempting to cross the continent from east to west. Edmund Kennedy led an expedition into what is now far-western Queensland in 1847 before being speared by Aborigines in the Cape York Peninsula in 1848.",

In [8]:
# Example usage
query1 = "What is the capital of United States of America?"
top_documents1 = keyword_search(query1)

In [9]:
top_documents1

[{'text': 'In 2013 longtime New Age author Marianne Williamson launched a campaign for a seat in the United States House of Representatives, telling "The New York Times" that her type of spirituality was what American politics needed. "America has swerved from its ethical center", she said. Running as an independent in west Los Angeles, she finished fourth in her district\'s open primary election with 13% of the vote. In early 2019, Williamson announced her candidacy for the Democratic Party nomination for president of the United States in the 2020 United States presidential election. A 5,300-word article about her presidential campaign in "The Washington Post" said she had "plans to fix America with love. Tough love". In January 2020 she withdrew her bid for the nomination.',
  'title': 'New Age',
  'url': 'https://en.wikipedia.org/wiki?curid=21742'},
 {'text': 'Governor Juan José Estrada, member of the Conservative Party, led a revolt against President José Santos Zelaya, member of t

# Search usinig Dense Retrieval and Vector Database

In [10]:
# Dense Retrieval function
def dense_retrieval(query, results_lang='en', num_results=3):

    nearText = {"concepts": [query]}
    properties = ["text", "title", "url", "_additional {distance}"]

    # To filter by language
    where_filter = {
        "path": ["lang"],
        "operator": "Equal",
        "valueString": results_lang
        }
    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText)
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    result = response['data']['Get']['Articles']
    return result

In [11]:
top_documents = dense_retrieval(query)

In [12]:
top_documents

[{'_additional': {'distance': -150.58865},
  'text': "Heights of mountains are typically measured above sea level. Using this metric, Mount Everest is the highest mountain on Earth, at . There are at least 100 mountains with heights of over above sea level, all of which are located in central and southern Asia. The highest mountains above sea level are generally not the highest above the surrounding terrain. There is no precise definition of surrounding base, but Denali, Mount Kilimanjaro and Nanga Parbat are possible candidates for the tallest mountain on land by this measure. The bases of mountain islands are below sea level, and given this consideration Mauna Kea ( above sea level) is the world's tallest mountain and volcano, rising about from the Pacific Ocean floor.",
  'title': 'Mountain',
  'url': 'https://en.wikipedia.org/wiki?curid=37754'},
 {'_additional': {'distance': -150.0647},
  'text': 'Until 1852, Kangchenjunga was assumed to be the highest mountain in the world, but ca

In [13]:
# Example usage
top_documents1 = dense_retrieval(query1)

In [14]:
top_documents1

[{'_additional': {'distance': -148.4822},
  'text': "In 1785, the assembly of the Congress of the Confederation made New York City the national capital shortly after the war. New York was the last capital of the U.S. under the Articles of Confederation and the first capital under the Constitution of the United States. New York City as the U.S. capital hosted several events of national scope in 1789—the first President of the United States, George Washington, was inaugurated; the first United States Congress and the Supreme Court of the United States each assembled for the first time; and the United States Bill of Rights was drafted, all at Federal Hall on Wall Street. In 1790, New York surpassed Philadelphia as the nation's largest city. At the end of that year, pursuant to the Residence Act, the national capital was moved to Philadelphia.",
  'title': 'New York City',
  'url': 'https://en.wikipedia.org/wiki?curid=645042'},
 {'_additional': {'distance': -147.14255},
  'text': "The Unit

# Search using Dense Retrieval, Rerank and Vector Database

In [15]:
def rerank_responses(query, responses, num_responses=2):
    reranked_responses = co.rerank(
        model = 'rerank-english-v2.0',
        query = query,
        documents = responses,
        top_n = num_responses,
    )
    return reranked_responses

In [16]:
texts = [result.get('text') for result in top_documents]
reranked_text = rerank_responses(query, texts)

In [17]:
for i, result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{result}")

i:0
RerankResult<document['text']: Until 1852, Kangchenjunga was assumed to be the highest mountain in the world, but calculations and measurements by the Great Trigonometrical Survey of India in 1849 showed that Mount Everest, known as Peak XV at the time, is actually higher. After allowing for further verification of all calculations, it was officially announced in 1856 that Kangchenjunga was the third highest mountain., index: 1, relevance_score: 0.92892635>
i:1
RerankResult<document['text']: A 1986 expedition led by George Wallerstein made an inaccurate measurement showing that K2 was taller than Mount Everest, and therefore the tallest mountain in the world. A corrected measurement was made in 1987, but by then the claim that K2 was the tallest mountain in the world had already made it into many news reports and reference works., index: 2, relevance_score: 0.876423>


In [18]:
texts = [result.get('text') for result in top_documents1]
reranked_text1 = rerank_responses(query1, texts)

In [19]:
for i, result in enumerate(reranked_text1):
    print(f"i:{i}")
    print(f"{result}")

i:0
RerankResult<document['text']: The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or America, is a country primarily in North America. It consists of 50 states, a federal district, five major unincorporated territories, nine Minor Outlying Islands, and 326 Indian reservations. It is the world's third-largest country by both land and total area. The United States shares land borders with Canada to its north and with Mexico to its south. It has maritime borders with the Bahamas, Cuba, Russia, and other nations. With a population of over 331 million, it is the most populous country in the Americas and the third most populous in the world. The national capital is Washington, D.C., and the most populous city and financial center is New York City., index: 1, relevance_score: 0.95719784>
i:1
RerankResult<document['text']: In 1785, the assembly of the Congress of the Confederation made New York City the national capital shortly after the war. New